In [1]:
def read_csv_list(my_list, my_tag):
    tag_name = my_tag
    file_name = tag_name + '_tag_link.csv'
    
    with open(file_name) as file:
        for line in file:
            read_data = line
            my_list.append(read_data[:-2])
        file.close()

my_tag = 'catstagram'
my_links = []
read_csv_list(my_links, my_tag)
print(my_links[0])
print(len(my_links))

https://www.instagram.com/p/BsdT6u3HF14/
3393


In [2]:
## 추출하고자 하는 내용은 다음과 같습니다.

## id ( 게시글 아이디 ) -> OK
## username ( 유저 아이디) -> OK
## date ( 게시 날짜 ) -> OK
## contents ( 글 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 ) -> OK
## comment_num ( 댓글 개수 ) -> OK

In [3]:
import requests
import re
import time
import json
import emoji
from bs4 import BeautifulSoup

In [4]:
## 찾아낸 데이터를 JSON 객체로 생성하는 함수를 정의합니다.
def data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num):
    single_data = {
        "find_tag" : my_tag,
        "id" : id,
        "username" : username,
        "date" : date,
        "contents" : contents,
        "hashtags" : hash_tags,
        "imagelinks" : final_image_link,
        "likes" : likes_num,
        "comments" : comments_num
    }
    return single_data

In [7]:
json_list = []
emoji_keys = emoji.UNICODE_EMOJI.keys()

for i in range(len(my_links)-1):
# for i in range(1):
    
    test_url = my_links[i]
    req = requests.get(test_url)
    html = req.text
    header = req.headers
    status = req.status_code
    soup = BeautifulSoup(html, 'html.parser')
    script_content = soup.find_all('script')

    ## 게시글 아이디를 링크로부터 가져와 찾아냅니다.
    id_p = re.compile("\/p\/(.*?)\/")
    id = id_p.findall(test_url)[0]

    ## 사용자 이름을 rel='canonical'에서 주소를 가져와 찾아냅니다.
    user_content = soup.find(rel="canonical")
    if user_content:
        username_pp = user_content.get('href')
        username_p = re.compile("instagram.com/(.*)/p")
        username = username_p.findall(str(username_pp))
        if len(username) > 0:
            username = username[0]
        else:
            username = ''


    ## 날짜를 header 파일에서 'Date' 부분을 가져와 찾아냅니다.
    date_p = re.compile("uploadDate\":\"(.{19})\"")
    date = date_p.findall(str(script_content))
    if len(date) > 0:
        date = date[0]
    else:
        date = ''
        

    ## 글 내용을 script 내용에서 caption 부분을 가져와 찾아냅니다.
    if script_content:   
        contents_p = re.compile("\"caption\":\"(.*?)\"")
        contents = contents_p.findall(str(script_content))
        
        if len(contents) > 0:
            contents = contents[0]
            contents = contents.encode('utf-8')
            contents = contents.decode('unicode_escape')
            contents = contents.encode('utf-8','ignore')
            contents = contents.decode('utf-8')
        else:
            contents = ''
            

    ## 해쉬태그를 property='instapp:hashtags' 에서 contetn= 이후 부분을 가져와 찾아냅니다.
    meta_content = soup.find_all(property = "instapp:hashtags")
    if meta_content:
        hash_tags_p = re.compile("content=\"(.*?)\"")
        emoji_hashtag = hash_tags_p.findall(str(meta_content))
        
        ## 해쉬태그에서 emoji 를 삭제하는 코드입니다.
        hash_tags = []
        for tag in emoji_hashtag:
            for e in emoji_keys:
                emoji_have = tag.find(e)

                if emoji_have > -1:
                    tag = tag.replace(e,'')
            hash_tags.append(tag)
            
        hash_tags = list(filter(None,hash_tags))      
    
    ## 해쉬태그 리스트가 완성되었으므로, 해당 해쉬태그가 본문에 작성되었다면 삭제해 줍니다.
    for tag in hash_tags:
        tag = '#' + tag
        contents = contents.replace(tag,"",1)

    contents = contents.replace('#','')
    contents = re.sub("\\\\u[0-9A-Fa-f]{4}", "", contents)
    contents = re.sub("[-()\"#/@;:<>{}`+=~|.!?,]", "", contents)
    contents = re.sub('\n', '', contents)
    contents = re.sub('\s+', ' ', contents)

    ## 이미지 링크를 display_resources 에서 가져와 찾아냅니다.
    ## 이미지가 N개일 때, 총 N+1 종류의 이미지 링크가 존재합니다.
    ## 확인해 보니 1번째와 2번째 이미지는 동일하며, 나머지는 서로 다른 이미지였습니다.
    ## 추정해 보면 1번째 이미지는 2번째 이후 이미지들을 대표하는 하나의 이미지라 할 수 있습니다.
    ## 각 이미지에 대해 서로 다른 resolution 으로 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
    ## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보이며, 이 중에서 원본 크기의 링크만 저장합니다.
    image_links_p = re.compile("\"display_resources\":\[.*?\]")
    image_links = image_links_p.findall(str(script_content))

    final_image_link = []
    single_image_link_p = re.compile("\"src\":\"(.*?)\"")
    for dif_img in image_links:
        link = single_image_link_p.findall(str(dif_img))[2]
        final_image_link.append(link)
        

    ## 이미지가 1개일 때는 대표 이미지만 존재하므로, 이에 대한 예외처리가 필요하다
    if len(final_image_link) != 1:
        final_image_link = final_image_link[1:]    

    ## like_num, comment_num 을 description 에서 찾아냅니다.
    likes_num_p = re.compile("\"description\":\"(.*?)Likes")
    likes_num = likes_num_p.findall(str(script_content))
    
    if len(likes_num) > 0:
        likes_num = likes_num[0]
        likes_num = re.sub('[,\s]', '', likes_num)
    else:
        likes_num = '0'
    

        
    comments_num_p = re.compile("\"description\":\".*?,(.*?)Comments")
    comments_num = comments_num_p.findall(str(script_content))

    if len(comments_num) > 0:
        comments_num = comments_num[0]
        comments_num = re.sub('[,\s]', '', comments_num)
    else:
        comments_num = '0'
    

    single_json = data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num)
    json_list.append(single_json)
    time.sleep(0.01)
    if i % 100 == 0:
        print(i, '번째 데이터')
        print(single_json)
        print('------------------')
        
print('Crawling 완료!')

0 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdT6u3HF14', 'username': 'luigibollini', 'date': '2019-01-10T14:49:36', 'contents': 'I am so proud to share this picture from Georgia US A beautiful cat wearing my pet harness Thank you Luigibollini Etsy shop s animal ', 'hashtags': ['world', 'catofinstagram', 'catoftheday', 'kitty', 'cat', 'catsofinstagram', 'food', 'travel', 'kittens', 'pet', 'kitten', 'instagram', 'katze', 'cats', 'meow', 'catsagram', 'art', 'cute', 'catlover', 'bar', 'love', 'animal', 'instacat', 'neko', 'likeforlike', 'of', 'catlovers', 'bhfyp', 'animals', 'catstagram'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/5e80a1a95dc8d3673b88e4be8a0640ff/5CD23BDF/t51.2885-15/e35/47692178_323940341795873_5756184700393723862_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': '14', 'comments': '0'}
------------------


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: DeprecationWarning: invalid escape sequence '\/'


100 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdT2FZHMrL', 'username': 'necoumetamaki', 'date': '2019-01-10T14:48:58', 'contents': 'おやすみなさい〜〜 s_of_instagram tagram cat catcats', 'hashtags': ['cat', 'ilovemycat', '保護猫', 'ふわもこ部', 'にゃんこ部', 'もふもふ', 'ネコ', '三毛猫', 'たまちゃん', '猫', '野良猫', 'のらねこ', 'cats_of_instagram', 'catsofinstagram', 'にゃんこ', 'みけねこ', 'cats', 'ねこ', 'のらねこ部', 'catstagram', '猫好きな人と繋がりたい'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/e2b3524a443bfff4ca399f767de4d3c3/5CBD82D0/t51.2885-15/e35/47692784_2394556257235348_8884669422062675292_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': '12', 'comments': '0'}
------------------
200 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdQwAGFEJN', 'username': 'healtyanimals_cat', 'date': '2019-01-10T14:21:56', 'contents': '大人になって儚げな表情も出来るようになったよー ', 'hashtags': ['ねこのいる暮らし', 'ヘルシーアニマルズ', 'ねこすたぐらむ', 'ねこ好き', 'ネコ', 'ソマリ', 'catstagram', 'にゃんすたぐらむ', 'ねこ部', 'ねこ', '猫'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/6b

1100 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdTwONhQK_', 'username': 'mauro_aspro', 'date': '2019-01-10T14:48:10', 'contents': '•Ερμής ', 'hashtags': ['cat', 'catsofinstagram', 'cateye', 'cats_of_instagram', 'yellow', 'eyes', 'catstagram'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/f81535a5e05329ab2a5153626b7be778/5CC8295C/t51.2885-15/e35/49354191_293906831269200_6485562050261610302_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': '34', 'comments': '5'}
------------------
1200 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdRyizBELU', 'username': 'sonsen83', 'date': '2019-01-10T14:31:01', 'contents': 'Pues para mi mejor que mami no esté en casa así puedo moder tranquilamente a Carri sin que mami venga a reñirme Ya me estoy imaginando el próximo mordisco Qué divertido será love cats puur purr kitty love lovecat lovelycat lovemycat lovemycatsomuch carri cats catstagram catsofinstagram catsareawesome crazycats weirdcats love cats animal whitecat catoftheday puur 

2200 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdNLIfnjxl', 'username': 'katzen_fan1234', 'date': '', 'contents': '', 'hashtags': ['cats_of_instagram', 'katzenbabys', 'cats', 'katzenbaby', 'katzen', 'catsofinstagram', 'susi', 'katzenliebhaber', 'julia', 'katzenfan', 'katzenfoto', 'katze', 'cat', 'cats_of_world', 'katzenliebe', 'catstagram', 'katzenliebe️', 'catlover'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/7f920adb0de0c647067ab04aecba6984/5CB7F952/t51.2885-15/e35/49759339_568680450260335_2531800197458249416_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': 0, 'comments': 0}
------------------
2300 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdUKHfBgYY', 'username': 'clinicameau', 'date': '2019-01-10T14:51:42', 'contents': 'Ainda há muita dúvida entre os donos de gatinhos se é certo ou não aparar o bigode deles Existe um mito antigo de que cortar o bigode do gato pode até mesmo matálo por ser “o equilíbrio do animal” Mas o importante é manter o bigode dos fel

3100 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdQ2NXnaHY', 'username': 'hazzi_s2', 'date': '', 'contents': '', 'hashtags': ['집사님들', '인천', '고양이', 'hazzigram', '고양이그램', '야옹이', '부평', '야옹', 'catstagram', '뽑기', '모자', '냥이스타그램', '토끼', '3500원', '토깽이', '하찌그램', '하찌', '야옹스타그램', '냥귀여움', '맞팔환영', 'cat', 'hazzi', '토끼모자', '아트박스'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/36cbf0a4c3dc602be1fbb2ed15fcbaa6/5CB94C2C/t51.2885-15/e35/47584087_309737962989536_7263628853033831523_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com'], 'likes': 0, 'comments': 0}
------------------
3200 번째 데이터
{'find_tag': 'catstagram', 'id': 'BsdSW8Vgz9r', 'username': 'meinmiezenmobil', 'date': '2019-01-10T14:35:59', 'contents': '❤❤❤ ANTON ❤❤❤ ', 'hashtags': ['tabbycatworld', 'kot', 'dreamjob', 'catsofinstagram', 'catoftheday', 'gato', 'chat', 'ig_cats', 'meowbox', 'instacat', 'potd', 'catsitting', 'catpic', 'tabbycatworlddomination', 'kucing', 'meowy', 'catsplanet', 'felino', 'ig_catphoto', 'catworld', 'bestmeow'

In [8]:
## 생성한 데이터를 파일 형태로 저장합니다.

def save_jsonlist_json(myjson):
    file_name = my_tag + '_rawdata.json'
    with open(file_name, "w") as file:
        for line in myjson:
            file.write(str(line) + ',\n')

save_jsonlist_json(json_list)

In [9]:
## 앞으로 해야할 일

## 1. Like 와 Comment 형태를 1,000 등의 string 에서 1000 의 int 로 변환 저장

## !. time.sleep 을 주지 않고 크롤링 하니 300번째 정도에서 104 error 가 발생하였다
##    밴인지는 정확하게 모르겠지만, 몇번 더 확인해볼 필요가 있다. 0.1 sleep 에서는 정상작동 하였다.
##    여기서 점차 sleep time 을 줄여가며 속도를 올릴 필요가 있다